<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Red%20convolucional.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tensorflow.keras as keras
import sklearn as sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import MaxPooling1D
from keras.layers import Conv1D
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras import layers
from google.colab import files
from sklearn.metrics import roc_auc_score
from sklearn import metrics
!pip install talos
import talos

     |████████████████████████████████| 26.6MB 166kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.2.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [29]:
# CARGAMOS LOS CSV
url_train = 'https://raw.githubusercontent.com/JulioCastillo1/TP1-Datos/master/train.csv'
url_test = 'https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test.csv'

train_df = pd.read_csv(url_train)
test_df = pd.read_csv(url_test)
train_df = train_df.drop(['keyword', 'location','id'], axis = 1)
train_df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [30]:
train_df.shape

(7613, 2)

In [31]:
x_train, x_validation, y_train, y_validation = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
x_train = train_df['text']
y_train = train_df['target']

In [32]:
def modelardo(x_train,y_train,x_test,y_test,params):
  vectorizer = CountVectorizer()
  vectorizer.fit(x_train)

  x_train = vectorizer.transform(x_train)
  x_test = vectorizer.transform(x_test)
  input_dim = x_train.shape[1]
  model = Sequential()
  model.add(layers.Dense(params['first neuron units'],input_dim=input_dim,activation=params['first neuron activation']))
  model.add(layers.Dense(1,activation = 'sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = model.fit(x_train,y_train,epochs = params['epochs'],verbose = 0,validation_data = (x_test,y_test),batch_size=params['batch size'])
  return out,model


In [33]:
x_train.shape
#t = talos.Scan(x_train,y_train,params=p,model=modelardo,experiment_name='diabetes')
#Este proceso tarda 1,5hs aprox en colab. Y tiene muy poquitos hiperparámetros para tunear

(7613,)

In [34]:
#experiment_data = t.data
#experiment_data.head()

In [35]:
#experiment_data['val_accuracy'].idxmax()

In [36]:
#experiment_data.iloc[205]

In [37]:
#vectorizer = CountVectorizer()
#vectorizer.fit(train_df['text'])

#data_train = vectorizer.fit_transform(train_df['text']).toarray()


In [38]:
# Función con el modelo para pasar por parámetro a talos.

def modelo(x_train,y_train,x_test,y_test,params):
  tokenizer = Tokenizer(num_words=2000)
  tokenizer.fit_on_texts(x_train)

  train_sequences = tokenizer.texts_to_sequences(x_train)
  validation_sequences = tokenizer.texts_to_sequences(x_validation)
  train_data = pad_sequences(train_sequences, maxlen=100)
  validation_data = pad_sequences(validation_sequences, maxlen=100)


  #input_dim = x_train.shape[1]

  best_model = Sequential()
  best_model.add(layers.Embedding(2000, 100, input_length=100))
  best_model.add(Conv1D(params['neuron_units_1'], 5, activation = params['activation']))
  best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(Conv1D(512, 5, activation='relu'))
  #best_model.add(layers.GlobalAveragePooling1D())
  #best_model.add(MaxPooling1D(5))
  #best_model.add(layers.Dropout(0.25))
  #best_model.add(Flatten())
  #best_model.add(LSTM(64))
  best_model.add(layers.Dense(params['neuron_units_2'], activation = params['activation']))
  #best_model.add(layers.Dense(128 ,activation='relu'))
  best_model.add(layers.Dense(units = 1, activation='sigmoid'))
  #best_model.add(layers.Dense(1,activation = 'sigmoid'))
  best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  out = best_model.fit(train_data,y_train,epochs = params['epochs'],verbose = 1,validation_data = (validation_data,y_validation),batch_size=params['batch size'])
  return out, best_model

#loss, accuracy = best_model.evaluate(train_data, y_train, verbose=False)
#print("Training Accuracy: {:.4f}".format(accuracy))
#loss, accuracy = best_model.evaluate(validation_data, y_validation, verbose=False)
#print("Testing Accuracy:  {:.4f}".format(accuracy))

p = {
    'neuron_units_1': [20,32,64,128],
    'neuron_units_2': [20,32,64,128],
    'activation' : ['relu','elu','softmax','tanh','exponential'],
    'epochs' : [1,2,3,4,5,10],
    'batch size': [5,100,5],
}

#t = talos.Scan(x_train, y_train, params = p, model = modelo, experiment_name = 'Convo hyperparam tuning')

In [39]:
experiment_data = t.data
experiment_data.head()

,round_epochs,val_loss,val_accuracy,loss,accuracy,activation,batch size,epochs,neuron_units
0,1,0.390322,0.842416,0.513121,0.751173,relu,5,1,20
1,1,0.394219,0.833224,0.508991,0.750985,relu,5,1,32
2,1,0.382724,0.836507,0.506041,0.750985,relu,5,1,64
3,1,0.405141,0.842416,0.513932,0.750798,relu,5,1,128
4,1,0.378328,0.848982,0.507067,0.751736,relu,5,1,256


In [40]:
experiment_data['val_accuracy'].idxmax()

356

In [41]:
experiment_data.iloc[205]

round_epochs           5
val_loss         0.36888
val_accuracy    0.848982
loss            0.295006
accuracy         0.88009
activation           elu
batch size             5
epochs                 5
neuron_units          32
Name: 205, dtype: object

Dalos experimentales:

round_epochs           5
val_loss         0.36888
val_accuracy    0.848982
loss            0.295006
accuracy         0.88009
activation           elu
batch size             5
epochs                 5
neuron_units          32
Name: 205, dtype: object

In [42]:
# Divide el set en train y validation, y entrena con el modelo tomando los hiperparámetros sugeridos

x_train, x_test, y_train, y_test = train_test_split(train_df['text'],train_df['target'],random_state=7,test_size = 0.20)
  
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(x_train)

train_sequences = tokenizer.texts_to_sequences(x_train)
validation_sequences = tokenizer.texts_to_sequences(x_validation)
train_data = pad_sequences(train_sequences, maxlen=100)
validation_data = pad_sequences(validation_sequences, maxlen=100)


#input_dim = x_train.shape[1]

best_model = Sequential()
best_model.add(layers.Embedding(2000, 100, input_length=100))
best_model.add(Conv1D(32, 5, activation='elu'))
best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='relu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(Conv1D(512, 5, activation='relu'))
#best_model.add(layers.GlobalAveragePooling1D())
#best_model.add(MaxPooling1D(5))
#best_model.add(layers.Dropout(0.25))
#best_model.add(Flatten())
#best_model.add(LSTM(64))
best_model.add(layers.Dense(32 ,activation='elu'))
#best_model.add(layers.Dense(128 ,activation='relu'))
best_model.add(layers.Dense(1,activation='sigmoid'))
#best_model.add(layers.Dense(1,activation = 'sigmoid'))
best_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
out = best_model.fit(train_data,y_train,epochs=5,verbose = 1,validation_data = (validation_data,y_validation),batch_size=5)

Train on 6090 samples, validate on 1523 samples
Epoch 1/5
6090/6090 [==============================] - 10s 2ms/step - loss: 0.5599 - accuracy: 0.7043 - val_loss: 0.4773 - val_accuracy: 0.7748
Epoch 2/5
6090/6090 [==============================] - 10s 2ms/step - loss: 0.4077 - accuracy: 0.8236 - val_loss: 0.4464 - val_accuracy: 0.8076
Epoch 3/5
6090/6090 [==============================] - 10s 2ms/step - loss: 0.3626 - accuracy: 0.8475 - val_loss: 0.4524 - val_accuracy: 0.8063
Epoch 4/5
6090/6090 [==============================] - 10s 2ms/step - loss: 0.3310 - accuracy: 0.8576 - val_loss: 0.4695 - val_accuracy: 0.8063
Epoch 5/5
6090/6090 [==============================] - 10s 2ms/step - loss: 0.3000 - accuracy: 0.8759 - val_loss: 0.4940 - val_accuracy: 0.8011


In [43]:
prediction=best_model.predict_classes(validation_data, batch_size=5)
score = roc_auc_score(y_validation, prediction)
score

0.7954128375983803

In [44]:
accuracy=metrics.accuracy_score(y_validation, prediction)
accuracy

0.8010505581089954

In [45]:
#Reentrena con el set completo y realiza predicción.

#set_sequences = tokenizer.texts_to_sequences(train_df['text'])
#set_data = pad_sequences(set_sequences, maxlen=100)

#out = best_model.fit(set_data,train_df['target'],epochs = 5,verbose = 1,batch_size=62)

#out = best_model.fit(data_train,train_df['target'],epochs = 5,verbose = 1,batch_size=62)
#data_test = vectorizer.transform(test_df['text']).toarray()
test_prediction = tokenizer.texts_to_sequences(test_df['text'])
prediction_data = pad_sequences(test_prediction, maxlen=100)

prediction2=best_model.predict_classes(prediction_data, batch_size=5)
prediction2

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

In [46]:
# Genera archivo submit.

test_df['target'] = prediction2
export = pd.DataFrame()
export['id'] = test_df['id']
export['target'] = test_df['target']
export = export.set_index('id')
export.to_csv('submit.csv')
#files.download('submit.csv')